## Example python code for using `delta-processor-axis`

We describe the whole process from the `3D model URL` to `medial axis` using docker.  
Pypi packages `requests` and `docker` are required to run this sample project.  
Note that the model URLs might not be available at the moment you run the code.  
You can reference this code to build an intergrated system for medial axis problem.

In [1]:
import time
import os
import shutil
import uuid
import warnings

import requests
import docker

In [2]:
def filesize(filepath):
    num = os.path.getsize(filepath)
    suffix = 'B'
    for unit in ['','K','M','G','T','P','E','Z']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Y', suffix)


def create_workspace_and_download_model(model_url):
    """ Create a temporal workspace for medial axis and download the model to the workspace.
    """
    # Generate a working directory using uuid (random hex generation)
    workspace = uuid.uuid4().hex[0:12]
    os.mkdir(workspace)
    
    # Get filename and path
    filename = model_url.split('/')[-1]
    filepath = os.path.join(workspace, filename)
    
    # Download the model
    with requests.get(model_url, stream=True) as r:
        with open(filepath, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    
    return workspace, filename


def medial_axis(dirname, input_filename, running_args=''):
    """ Execute medial axis from the local 3d model file.
    """
    # Map the workspace directory to the /data directory of the container using volume
    volume_path = dirname
    if volume_path[0] != '/':
        volume_path = os.path.join(os.getcwd(), volume_path)

    start_time = time.time()

    # Run docker container
    client.containers.run(
        image='kaistgclab/delta-processor-axis',
        auto_remove=True,
        volumes={volume_path: {'bind': '/data', 'mode': 'rw'}},
        command=f'-i {input_filename} {running_args}'
    )

    time_consumed = round(time.time() - start_time)

    return {
        'output': os.path.join(dirname, 'axis.off'),
        'time_consumed': time_consumed,
    }

### Check docker image exists

In [3]:
client = docker.from_env()
image_exists = False

for image in client.images.list():
    for tag in image.tags:
        if 'kaistgclab/delta-processor-axis:' in tag:
            image_exists = True
            break
            
if not image_exists:
    warnings.warn('Docker image "kaistgclab/delta-processor-axis:*" not exists on your machine\n'
                  'Please build the image first before trying this script.')
else:
    print('Docker image exists.')

Docker image exists.


### Run medial axis

In [4]:
MODEL_DOWNLOAD_URL_1 = 'http://research.prev.kr/delta/models/knot.pts'

# Download the first model file for medial axis
workspace, input_filename = create_workspace_and_download_model(MODEL_DOWNLOAD_URL_1)

# Run medial axis using docker
result = medial_axis(workspace, input_filename)

print(f'- Output file is located in "{result["output"]}"')
print(f'- Size of the output file is {filesize(result["output"])}')
print(f'- Time consumed: {result["time_consumed"]} seconds')

- Output file is located in "f9d1f0ac147c/axis.off"
- Size of the output file is 129.3KB
- Time consumed: 1 seconds


In [5]:
MODEL_DOWNLOAD_URL_2 = 'http://research.prev.kr/delta/models/horse.off'

# Download the second model file for medial axis
workspace, input_filename = create_workspace_and_download_model(MODEL_DOWNLOAD_URL_2)

# Run medial axis using docker
result = medial_axis(workspace, input_filename)

print(f'- Output file is located in "{result["output"]}"')
print(f'- Size of the output file is {filesize(result["output"])}')
print(f'- Time consumed: {result["time_consumed"]} seconds')

- Output file is located in "6e52b6fd994a/axis.off"
- Size of the output file is 94.0MB
- Time consumed: 36 seconds
